In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
import datetime as dt

In [2]:
with ZipFile()
#Import the data saved in .csv format
data= pd.read_csv('CoVid19/covid_19_data.csv',index_col=[0])
data.tail(5)

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
SNo,,,,,,,
4509,03/09/2020,Northern Territory,Australia,2020-03-06T04:33:03,0.0,0.0,0.0
4510,03/09/2020,"Lackland, TX (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0
4511,03/09/2020,"Montgomery County, TX",US,2020-03-07T19:53:02,0.0,0.0,0.0
4512,03/09/2020,"Omaha, NE (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0
4513,03/09/2020,"Travis, CA (From Diamond Princess)",US,2020-02-24T23:33:02,0.0,0.0,0.0


In [3]:
#Explore data types and sums of the different types
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4513 entries, 1 to 4513
Data columns (total 7 columns):
ObservationDate    4513 non-null object
Province/State     2912 non-null object
Country/Region     4513 non-null object
Last Update        4513 non-null object
Confirmed          4513 non-null float64
Deaths             4513 non-null float64
Recovered          4513 non-null float64
dtypes: float64(3), object(4)
memory usage: 282.1+ KB


In [4]:
#Fill NaN, None,'From Diamond Princess' values in Province/State with corresponding country names.
#For these observations, we dont know the exact location of the observation,we aggregate on country level
data['Province/State'] = (data['Province/State'].replace(np.nan, data['Country/Region']).
                          replace('From Diamond Princess', data['Country/Region']).
                          replace('Grand Princess Cruise Ship', data['Country/Region']).
                          replace('None', data['Country/Region']))

In [5]:
#Extracting the state code
data['Province/State'] = (data['Province/State'].
                          astype(str).apply(lambda x: x.split(' (')[0]).
                          apply(lambda x: x.split(', ')[-1]))

data['Province/State'].tail()

SNo
4509    Northern Territory
4510                    TX
4511                    TX
4512                    NE
4513                    CA
Name: Province/State, dtype: object

In [6]:
#Create a dictionary of state name abbreviations
state_dict = {'D.C.':'District Columbia','AL':'Alabama','AK':'Alaska', 'AZ':'Arizona',
              'AR':'Arkansas','CA':'Califonia','CO':'Colorado','CT':'Connecticut',
              'FL':'Florida','GA':'Georgia','HI':'Hawaii','IL':'Illinois', 
              'IN':'Indiana','IA':'Iowa','KS':'Kansas','TX':'Texas', 'NE':'Nebraska',
              'KY':'Kentucky','LA':'Louisiana', 'ME':'Maine','MD':'Maryland',
              'MA':'Masachusetts', 'MI':'Michigan', 'QC':'Quebec','NV':'Nevada','TN':'Tennessee',
              'SC':'South Carolina', 'UT':'Utah','MO':'Missouri','OR ':'Oregon',
              'MN':'Minnesota','NH':'New Hmapshire','NJ':'NewJersey','NM':'New Mexico', 
              'NY':'New York','NC':'North Carolina', 'OH':'Ohio','OK':'Okloahome',
              'OR':'Oregon','PA':'Pennsylvia', 'VT':'Vermont','VA':'Virginia',
              'WA':'Washington','WV':'West Virginia','WI':'Wisconsir','WY':'Wyoming',
              'ON':'Oklahoma','RI':'Rhode Island','Nigeria':'Lagos'}


#Replace state abbreciations with fullnames 
data['Province/State'] = data['Province/State'].replace(state_dict)

#Create name abbrevaition dict
country_dict = {'US': 'United States','UK': 'United Kingdom'}

#Map dictionary of country names
data['Country/Region'] = (data['Country/Region'].replace(country_dict)
                          .replace('Mainland China','China'))
data.shape

(4513, 7)

In [7]:
#Retain only observations with confirmed cases
data = data[data['Confirmed'] > 0]
print('\n')
print(f'Shape of data:', data.shape)
print('\n')
data.tail()



Shape of data: (4431, 7)




,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
SNo,,,,,,,
4503,03/09/2020,Pennsylvia,United States,2020-03-06T16:13:15,1.0,0.0,0.0
4504,03/09/2020,Tennessee,United States,2020-03-05T23:33:03,1.0,0.0,0.0
4505,03/09/2020,Califonia,United States,2020-03-06T20:13:14,1.0,0.0,0.0
4506,03/09/2020,Ukraine,Ukraine,2020-03-03T15:33:02,1.0,0.0,0.0
4507,03/09/2020,Vatican City,Vatican City,2020-03-06T15:43:02,1.0,0.0,0.0


## Remove Non Country/Region Observations 

In [8]:
#Certains observations made outside particular country/region such as those from boat cruise are excluded from this analysis
data = data[data['Country/Region']!='Others']
data.shape

(4399, 7)

In [9]:
cols = ['ObservationDate', 'Province/State', 'Country/Region',
       'Confirmed', 'Deaths', 'Recovered', 'Last Update']

#Rearranging the columns names
data = data[cols]
data.columns

Index(['ObservationDate', 'Province/State', 'Country/Region', 'Confirmed',
       'Deaths', 'Recovered', 'Last Update'],
      dtype='object')

In [ ]:
#Save cleaned data for vizualization
date = pd.Timestamp.today().strftime(f'%d-%m-%Y')

data.to_csv(f'CoVid19/covid_data_{date}_cleaned.csv', index=True, header=True)

In [ ]:
data.head()

In [ ]:
#Observation with the maximum number of confirmed cases
data[data['Confirmed']==data[data['Country/Region']=='China']['Confirmed'].max()]

In [ ]:
data[data['Country/Region']=='Others']